In [11]:
import numpy as np
import os
import pandas as pd
import glob
import datetime
import json
import math
from operator import itemgetter

In [12]:
os.chdir('C:/Users/rodri/Desktop/NCFS2/DistComp/proj2')

In [13]:
#extension = 'csv'
#all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [14]:
#combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

In [15]:
csv_file = "C:/Users/rodri/Desktop/NCFS2/DistComp/proj2/combined_csv.csv"
df= pd.read_csv(csv_file,sep=',')

In [16]:
df.head()

,Date/Time,Lat,Lon,Base
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512


In [19]:
df.rename(columns={
'Date/Time':'pickup_datetime',
'Lat':'lat',
'Lon':'lon',
'Base':'pickup_base',
},inplace=True)

In [20]:
df.head()

,pickup_datetime,lat,lon,pickup_base
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512


In [21]:
import geopandas as gpd

apr14_gdf = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df.lon, df.lat))
apr14_gdf.head()

,pickup_datetime,lat,lon,pickup_base,geometry
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512,POINT (-73.95490 40.76900)
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512,POINT (-74.03450 40.72670)
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512,POINT (-73.98730 40.73160)
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512,POINT (-73.97760 40.75880)
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512,POINT (-73.97220 40.75940)


In [22]:
apr14_gdf.to_csv(r"C:/Users/rodri/Desktop/NCFS2/DistComp/proj2/FinalCleanedData.csv",index=False)

In [23]:
apr14_gdf_test = apr14_gdf[:1000].copy()

In [24]:
def coord_to_zone(point): 
    ''' Given a point, identifies the locationID that it's contained in'''
    
    # Vector indicating which of the zones contains the given point
    contained_in_zone = zones_gdf['geometry'].contains(point)
    
    # Point is not contained in any of the zones. LocationID = 265 
    if max(contained_in_zone) == 0: 
        locationID = 265
    
    # Finds the zone the point is contained in (where vector = True)
    # Assumes a point does not land in multiple zones
    else: 
        # Relies on truthy/ falsey (i.e. 1/0) conversion
        location_index = np.argmax(contained_in_zone) 
        locationID = zones_gdf.iloc[location_index]['location_i']
        
    return int(locationID)

In [25]:
apr14_gdf_test['locationID'] = apr14_gdf_test['geometry'].apply(lambda x: coord_to_zone(x))
apr14_gdf_test.head()

NameError: name 'zones_gdf' is not defined